In [18]:
# ! pip install neo4j

In [39]:
import pandas as pd

In [ ]:
#LOAD CSV WITH HEADERS FROM 
#"file:///Datos.csv" AS row
#MERGE (p:Producto {nombre_producto:row.PRODUCTO})
#MERGE (t:Tienda {nombre_tienda:row.RAZON_SOCIAL, estado:row.ENTIDAD, municipio:row.MUNICIPIO, giro_comercial:row.GIRO_COMERCIAL})
#CREATE (p)-[v:SE_VENDE_EN {cumplimiento:row.CUMPLIMIENTO, norma:row.NOM}]->(t)

In [16]:
# import the neo4j driver for Python
from neo4j import GraphDatabase

# Database Credentials
uri             = "neo4j://localhost:7687"
userName        = "neo4j"
password        = "12345"

# Connect to the neo4j database server
graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))
graphDB_Session = graphDB_Driver.session()

In [17]:
cqlNodeQuery = "MATCH (n) RETURN n LIMIT 10"
nodes = graphDB_Session.run(cqlNodeQuery)
for node in nodes:
    print(node)

<Record n=<Node id=2725 labels=frozenset({'Producto'}) properties={'nombre_producto': 'ATUN'}>>
<Record n=<Node id=2726 labels=frozenset({'Tienda'}) properties={'giro_comercial': 'ABARROTES', 'estado': 'AGUASCALIENTES', 'municipio': 'AGUASCALIENTES', 'nombre_tienda': 'ABARROTES - JOSE ALVARO NUNEZ GUZMAN'}>>
<Record n=<Node id=2727 labels=frozenset({'Producto'}) properties={'nombre_producto': 'FRIJOLES'}>>
<Record n=<Node id=2728 labels=frozenset({'Tienda'}) properties={'giro_comercial': 'ABARROTES', 'estado': 'AGUASCALIENTES', 'municipio': 'AGUASCALIENTES', 'nombre_tienda': 'ABARROTES - JOSE ALVARO NU�EZ GUZMAN'}>>
<Record n=<Node id=2729 labels=frozenset({'Producto'}) properties={'nombre_producto': 'VELADORAS  Y CERAS'}>>
<Record n=<Node id=2730 labels=frozenset({'Producto'}) properties={'nombre_producto': 'BRANDY'}>>
<Record n=<Node id=2731 labels=frozenset({'Tienda'}) properties={'giro_comercial': 'RESTAURANT-BAR', 'estado': 'AGUASCALIENTES', 'municipio': 'AGUASCALIENTES', 'nombre_

In [33]:
def ingresa_compra(usuario, tienda, estado, municipio, giro, lista_productos, lista_precios, lista_cantidades):
    assert len(lista_productos) == len(lista_precios), 'Lista de productos y precios distinta'
    assert len(lista_productos) == len(lista_cantidades), 'Lista de productos y cantidades distinta'
    consulta = '''MERGE (u:Usuario {nombre:'%s'}) 
CREATE (u)-[:HACE]->(c:Compra {hora:time({timezone: 'America/Mexico_City'})})
MERGE (t:Tienda {nombre_tienda:'%s', estado:'%s', municipio:'%s', giro_comercial:'%s'})
CREATE (c)-[:COMPRO_EN]->(t)
MERGE (p:Producto {nombre_producto:'%s'})
CREATE (c)-[:COMPRO {precio:%.2f, cantidad:%d}]->(p)''' % (usuario.upper(), tienda.upper(), estado.upper(), municipio.upper(), giro.upper(), lista_productos[0].upper(), lista_precios[0], lista_cantidades[0])
    
    l = len(lista_productos)
    if l > 1:
        for i in range(l-1):
            consulta += '''
MERGE (p%d:Producto {nombre_producto:'%s'})
CREATE (c)-[:COMPRO {precio:%.2f, cantidad:%d}]->(p%d)''' % (i, lista_productos[i+1].upper(), lista_precios[i+1], lista_cantidades[i+1], i)
    
    graphDB_Session.run(consulta)

In [34]:
ingresa_compra('Huguito', 'C&A DE MEXICO S. DE R.L.', 'AGUASCALIENTES', 'AGUASCALIENTES', 'TIENDA DEPARTAMENTAL', ['CHAMARRA', 'ROPA CABALLERO'], [500, 400], [2,1])
ingresa_compra('Daniel', 'TIENDAS SORIANA S.A. DE C.V. SUCURSAL GERANEOS', 'BAJA CALIFORNIA', 'ENSENADA', 'TIENDA DE AUTOSERVICIO', ['ATUN', 'PRODUCTOS ALIMENTICIOS'], [6.99, 25.87], [12, 5])
ingresa_compra('Daniel', 'LOB FOOTWEAR  S.A. DE C.V.', 'CAMPECHE', 'CAMPECHE', 'ZAPATERIA', ['ZAPATOS'], [300.50], [1])
ingresa_compra('Daniel', 'TATIANA�S CURIOS', 'BAJA CALIFORNIA', 'ENSENADA', 'ARTESANIAS', ['JOYERIA'], [1560.50], [1])
ingresa_compra('Huguito', 'SUBURBIA S. DE R.L. DE C.V.', 'DISTRITO FEDERAL', 'TLALPAN', 'TIENDA DEPARTAMENTAL', ['PANTALON', 'PLAYERA', 'ZAPATOS'], [100,200,300], [3,2,1])
ingresa_compra('Kike', 'PERFUMERIA DAISY  Y/O ADRIAN LEJARAZO GAONA', 'DISTRITO FEDERAL', 'CUAUHTEMOC', 'PERFUMERIA', ['ARTICULOS PARA EL CABELLO', 'ARTICULOS DE BELLEZA'], [1265.99, 200.01], [1,2])
ingresa_compra('Carlos', 'GRUPO INDUSTRIAL CUADRITOS BIOTEK S.A. DE C.V.', 'GUANAJUATO', 'CELAYA', 'LECHE', ['LECHE', 'PRODUCTOS LACTEOS'], [12.70, 55.80], [1, 5])
ingresa_compra('Ana', 'NETO TIENDAS SUPER PRECIO S.A DE C.V.', 'MEXICO', 'NAUCALPAN DE JUAREZ', 'ABARROTES', ['MAYONESA', 'SOPA DE PASTA', 'VERDURAS ENLATADAS'], [15.50, 25.60, 33.45], [3, 4, 1])
ingresa_compra('Juan', '7 ELEVEN MEXICO S.A. DE C.V.', 'MEXICO', 'NAUCALPAN DE JUAREZ', 'TIENDAS DE CONVENIENCIA', ['ENSALADAS', 'FRIJOLES', 'MAYONESA', 'PAN', 'SALSA', 'AGUA PURIFICADA'], [15.50, 25.60, 33.45, 30, 50.40, 60.49], [1, 2, 1, 3, 1, 12])


In [58]:
def consulta_producto_estado(producto, estado):
    consulta = '''MATCH (p:Producto {nombre_producto:'%s'})-[:SE_VENDE_EN]->(t:Tienda {estado:'%s'})
RETURN t.nombre_tienda, t.municipio''' % (producto.upper(), estado.upper())
    consulta = pd.DataFrame(graphDB_Session.run(consulta))
    if len(consulta) == 0:
        return 'No hay lugares en %s donde puedas encontrar %s' % (estado.upper(), producto.upper())
    else:
        consulta.columns = ['Tienda', 'Municipio']
    return consulta

In [125]:
consulta_producto_estado('frijoles', 'distrito federal')

,Tienda,Municipio
0,NUEVA WAL MART DE MEXICO S. DE R.L. DE C.V. ...,CUAUHTEMOC
1,ABARROTES VINOS Y LICORES MONICA Y/O ANTONIO...,GUSTAVO A. MADERO
2,ABARROTES Y VINOS LA UNION Y/O NORMA CONTRER...,GUSTAVO A. MADERO
3,ABARROTES VINOS Y LICORES BALDERAS Y/O IRENE V...,CUAUHTEMOC
4,ABARROTES DEL SUR Y/O RAFAEL JIMENEZ CAMACHO,COYOACAN
5,TIENDAS EXTRA S.A. DE C.V.,CUAUHTEMOC
6,TIENDAS COMERCIAL MEXICANA S.A. DE C.V. (MEGA...,IZTAPALAPA
7,NUEVA WAL MART DE MEXICO S. DE R.L. DE C.V. ...,GUSTAVO A. MADERO
8,ABARROTES SIN NOMBRE,TLAHUAC
9,ABARROTES LA UNION Y/O RUTILIO BAUTISTA BAUTISTA,VENUSTIANO CARRANZA


In [55]:
def consulta_estado_tienda(estado, tienda):
    consulta = '''MATCH (p:Producto)-[v:SE_VENDE_EN]->(t:Tienda {estado:'%s', nombre_tienda:"%s"})
WHERE v.cumplimiento = 'NO CUMPLE'
RETURN p.nombre_producto, v.norma''' % (estado.upper(), tienda.upper())
    consulta = pd.DataFrame(graphDB_Session.run(consulta))
    if len (consulta) == 0:
        return 'Todos los productos de esta tienda cumplen con la norma.'
    else:
        consulta.columns = ['Producto', 'Norma Incumplida']
    return consulta

In [57]:
consulta_estado_tienda('distrito federal', 'TIENDAS SORIANA  S.A. DE c.v.')

,Producto,Norma Incumplida
0,JUGUETES,NOM-015-SCFI-2007
1,APARATOS ELECTRICOS,NOM-001-SCFI-1993
2,APARATOS ELECTRICOS,NOM-003-SCFI-2000
3,CARRIOLA,NOM-133/2-SCFI-1999
4,ANDADERAS,NOM-133/1-SCFI-1999
5,BEBIDAS,NOM-051-SCFI/SSA1-2010


In [65]:
def consulta_opciones(estado, producto):
    consulta = '''MATCH (p:Producto {nombre_producto:'%s'})-[v:SE_VENDE_EN]->(t:Tienda {estado:'%s'})
WHERE v.cumplimiento = 'CUMPLE'
RETURN t.nombre_tienda, t.municipio''' % (producto.upper(), estado.upper())
    consulta = pd.DataFrame(graphDB_Session.run(consulta))
    if len (consulta) == 0:
        return 'No hay opciones de %s en %s sin incumplimiento'  % (producto.upper(), estado.upper())
    else:
        consulta.columns = ['Tienda', 'Municipio']
    return consulta

In [127]:
consulta_opciones('distrito federal', 'licor')

,Tienda,Municipio
0,CADENA COMERCIAL OXXO S.A. DE C.V.,COYOACAN
1,VINOS Y LICORES LA BARRICA DEL SIGLO XXI Y/O M...,IZTACALCO
2,CADENA COMERCIAL OXXO S.A. DE C.V.,ALVARO OBREGON
3,7 ELEVEN MEXICO S.A. DE C.V. (TIENDA 987 AERO...,VENUSTIANO CARRANZA
4,SUPERANDA S.A. DE C.V.,COYOACAN
5,CADENA COMERCIAL OXXO S.A. DE C.V.,VENUSTIANO CARRANZA
6,CADENA COMERICAL OXXO S.A. DE C.V.,IZTAPALAPA
7,TIENDAS EXTRA S.A. DE C.V.,MIGUEL HIDALGO
8,ABARROTES VINOS Y LICORES VICK,ALVARO OBREGON
9,CADENA COMERCIAL OXXO S.A. DE C.V.,CUAUHTEMOC


In [114]:
def estados_incumplimineto(tipo,limit=32):
    assert tipo == 'Mayor' or tipo == 'Menor', 'Ingrese un tipo válido (Mayor / Menor)'
    consulta1 = '''MATCH (t:Tienda)
RETURN t.estado, COUNT(*)'''
    consulta2 = '''MATCH (p:Producto)-[v:SE_VENDE_EN {cumplimiento:'NO CUMPLE'}]->(t1:Tienda)
RETURN t1.estado, COUNT(*)'''
    consulta1 = pd.DataFrame(graphDB_Session.run(consulta1))
    consulta1.columns = ['Estado', 'Numero_tiendas']
    consulta2 = pd.DataFrame(graphDB_Session.run(consulta2))
    consulta2.columns = ['Estado', 'Numero_incumplimientos']
    consulta = consulta1.merge(consulta2, on='Estado', how='left')
    consulta.fillna(0, inplace=True)
    consulta['Tasa_Incumplimiento'] = 100*consulta['Numero_incumplimientos']/consulta['Numero_tiendas']
    if tipo == 'Mayor':
        consulta = consulta.sort_values('Tasa_Incumplimiento', ascending = False).head(limit).reset_index(drop=True)
    if tipo == 'Menor':
        consulta = consulta.sort_values('Tasa_Incumplimiento').head(limit).reset_index(drop=True)
    
    return consulta
    

In [116]:
estados_incumplimineto('Menor', 5)

,Estado,Numero_tiendas,Numero_incumplimientos,Tasa_Incumplimiento
0,SAN LUIS POTOSI,144,0.0,0.000000
1,AGUASCALIENTES,104,2.0,1.923077
2,QUERETARO,203,4.0,1.970443
3,MORELOS,197,4.0,2.030457
4,TLAXCALA,331,7.0,2.114804


In [117]:
def recomienda(usuario, producto):
    consulta ='''MATCH (u:Usuario)-[:HACE]->(co:Compra)-[:COMPRO]->(p:Producto)
WHERE u.nombre <> '%s' AND p.nombre_producto = '%s'
MATCH (co)-[c1:COMPRO]->(p1:Producto)
WHERE p1.nombre_producto <> '%s'
RETURN p1.nombre_producto''' % (usuario.upper(), producto.upper(), producto.upper())
    consulta = pd.DataFrame(graphDB_Session.run(consulta))
    if len (consulta) == 0:
        return 'No hay recomendaciones'
    else:
        consulta.columns = ['Productos']
    return consulta

In [124]:
recomienda('Daniel', 'ZAPATOS')

,Productos
0,PANTALON
1,PLAYERA
